## overview
1. Every spark application consists of a driver program that runs
the users main function and excutes various parallel operatoins on
a cluster.
2. RDD is a collection of elements partitioned across the nodes of
cluster that can be operated on in parallel.
3. User can ask Spark to persist an RDD in memory.
4. RDD automatically reconver from node failures
5. Shared variables can be used in parallel operations.
6. When spark runs a function in parallel as a set of tasks on 
different nodes, it ships a copy of each variable used in the 
function to each tasks.
7. Spark supports tow types of share variables: broadcast variables
    , which can be used to cache a value in mememory on all nodes,
    and accumulators, which are varibles that are only 'added' to
    such as counters and sums

In [2]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
## The first thing a Spark program must do is to create a SparkContext
## which tells Spark how to access a cluster. To create a SparkContext
## you first need to build a SparkConf object that contains information
## about your application
conf = SparkConf().setAppName("RDD programing guide").setMaster("local")
sc = SparkContext(conf=conf)

## RDD can be created in two ways
1. parallelize method on an existing itertable or collection in your
driver program
2. referencing a dataset in an external storage system such as shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat

In [5]:
## parallelize collections
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
## wec can perform parallel operation after distribute the collection
distData.reduce(lambda a, b: a + b)

15

In [6]:
## External DataSets
## Spark Supports text files, SequenceFiles, and any other Hadoop
## InputFormat
distFile = sc.textFile("README.md")

In [7]:
distFile.map(lambda s: len(s)).reduce(lambda a, b: a + b)

3706